In [ ]:
import torch
from torch_geometric.data import Batch
from torch_geometric.loader import DataLoader
import gpytorch

import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(119, 360)
        self.fc2 = nn.Linear(360, 180)
        self.fc3 = nn.Linear(180, 100)
        self.fc4 = nn.Linear(100, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x
model = NN()

In [ ]:
def train(train_loader, device, model, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    average_loss = running_loss / len(train_loader)
    return average_loss

def validate(validation_loader, device, model, criterion):
    model.eval()
    running_loss = 0.0
    validation_outputs = []
    validation_truth = []

    with torch.no_grad():
        for inputs, labels in validation_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            validation_outputs.append(outputs.detach().numpy())
            validation_truth.append(labels.detach().numpy())

    average_loss = running_loss / len(validation_loader)
    validation_outputs = np.concatenate(validation_outputs)
    validation_truth = np.concatenate(validation_truth)

    return average_loss, validation_outputs, validation_truth


def test(test_loader, device, model, criterion):
    model.eval()
    running_loss = 0.0
    test_outputs = []
    test_truth = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            test_outputs.append(outputs.detach().numpy())
            test_truth.append(targets.detach().numpy())

    average_loss = running_loss / len(test_loader)
    test_outputs = np.concatenate(test_outputs)
    test_truth = np.concatenate(test_truth)

    return average_loss, test_outputs, test_truth

In [ ]:
sampling_size = 6

best_vals =[]
best_models = []
best_tests = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_test_split_index = int(0.9 * len(data_list))

train_val_data = data_list[:train_test_split_index]

test_data = data_list[train_test_split_index:]
test_loader = Dataloader(test_data, batch_size=16, shuffle=False)

data_len = len(train_val_data)
num_batches = data_len // sampling_size
indices = np.arange(data_len)
for i in range(num_batches):
    best_val_loss = np.inf

    start_index = i * sampling_size

    val_indices = indices[start_index:start_index + sampling_size]

    train_indices = np.setdiff1d(indices, val_test_indices)

    train_data = [train_val_data[j] for j in train_indices]
    val_data = [train_val_data[j] for j in val_indices]

    train_loader = Dataloader(train_data, batch_size=16, shuffle=True)
    validation_loader = Dataloader(val_data, batch_size=16, shuffle=False)

    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                        factor=0.8, patience=5,
                                                        min_lr=0.0000001)
    criterion = nn.L1Loss()
    best_validation_loss = float('inf')

    for epoch in range(1, EPOCHS+1):

        model.train()
        loss = train(train_loader, device, model, optimizer, criterion)
        scheduler.step(loss)

        validation_loss, validation_output, validation_truth_temp = validate(validation_loader, device, model, criterion)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")
        if validation_loss < best_validation_loss:
            best_validation_loss = validation_loss
            best_model_state = copy.deepcopy(model.state_dict())
            best_model = model
            best_val_loss = validation_loss
            print(f"Best Validation Loss: {best_val_loss:.4f}")
model.load_state_dict(best_model_state)
test_loss, test_outputs, test_truth = test(test_loader, device, model, criterion)
print(f"Test Loss: {test_loss:.4f}")


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

evals = [(dtrain, 'train'), (dval, 'eval')]

num_boost_round = 100
early_stopping_rounds = 10
model = xgb.train(params, dtrain, num_boost_round, evals, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

y_pred = model.predict(dtest)
test_rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Test RMSE: {test_rmse:.4f}')